# Introduction

## Train Multiple Models

Now that we have done feature engineering on the dataset, the next step is to train the data on different regression models.
The models we will be using are the following:
- **Linear Regression**
- **Decision Tree**
- **Adaboost Regressor**
- **Random Forest**

For each model, there will be a training and parameter tuning phase, and a predictions phase.
Results of the predictions are then plotted and visualized.

# Setup
Let us import the required modules.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sys
import os
from math import sqrt
import matplotlib.pyplot as plt
import pickle

import project.src.feat_eng as fe
import project.src.visualization as viz
import project.src.evaluation as eva

import sklearn.model_selection as modsel
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

%matplotlib inline
sys.path.insert(0, os.path.abspath("../../"))
color = sns.color_palette()
pd.set_option("display.max_columns", 100) #

RNDSTATE = 28

## Load Data
Note that the dataset is already split into Train-Test sets.

In [2]:
engineered_dataset = fe.TrainTestSplit.from_csv_directory(dir_path="../data/lvl4_rfecv")

In [3]:
engineered_dataset.x_train.info()
engineered_dataset.y_train

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62090 entries, 0 to 62089
Data columns (total 26 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bathroomcnt                   62090 non-null  float64
 1   bedroomcnt                    62090 non-null  float64
 2   fireplacecnt                  62090 non-null  float64
 3   garagecarcnt                  62090 non-null  float64
 4   latitude                      62090 non-null  float64
 5   longitude                     62090 non-null  float64
 6   poolcnt                       62090 non-null  float64
 7   roomcnt                       62090 non-null  float64
 8   threequarterbathnbr           62090 non-null  float64
 9   unitcnt                       62090 non-null  float64
 10  numberofstories               62090 non-null  float64
 11  house_age                     62090 non-null  float64
 12  airconditioningtypeid_1.0     62090 non-null  float64
 13  b

array([-0.04060104,  0.00803361,  0.04714126, ..., -0.02798657,
        0.03477351,  0.00171659])

# Linear Regression Model

-------SCRIVERE QUALCOSA----------------

Linear Regression: Plain linear regression that minimizes the Mean Squared Error(MSE) cost function.

The model RMSE is significantly higher than MAE which suggests that the outliers are affecting the model's performance as RMSE punishes the model more for mispredicting outliers.
The K-Fold Cross Validation shows that the model's performance is highly volatile

In [4]:
linear_reg = LinearRegression()
linear_reg.fit(engineered_dataset.x_train, engineered_dataset.y_train)

LinearRegression()

In [5]:
linreg_train_pred = linear_reg.predict(engineered_dataset.x_train)
linreg_test_pred = linear_reg.predict(engineered_dataset.x_test)

linreg_train_eval = eva.Evaluation(y_real=engineered_dataset.y_train, y_pred=linreg_train_pred)
linreg_test_eval = eva.Evaluation(y_real=engineered_dataset.y_test, y_pred=linreg_test_pred)

In [6]:
print("Training:")
linreg_train_eval.print_eval()
print("Testing:")
linreg_test_eval.print_eval()

Training:
--------------Model Evaluations:--------------
Mean Absolute Error : 0.07041153246114866
Mean Squared Error : 0.028827755428801553
Root Mean Squared Error : 0.1697873830082835

Testing:
--------------Model Evaluations:--------------
Mean Absolute Error : 0.0721586173392606
Mean Squared Error : 0.030189761313624365
Root Mean Squared Error : 0.17375201096282128



# Decision Tree Regression Model

-------SCRIVERE QUALCOSA----------------

Decision Tree: Powerful model capable of finding complex nonlinear relationships in the data.
Random Forest: Train many Decision Tress on random subsets of the features via the bagging method (Ensemble Learning).

In [7]:
# just to check if they have the same size
engineered_dataset.x_train.info(verbose=True)
engineered_dataset.y_train

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62090 entries, 0 to 62089
Data columns (total 26 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bathroomcnt                   62090 non-null  float64
 1   bedroomcnt                    62090 non-null  float64
 2   fireplacecnt                  62090 non-null  float64
 3   garagecarcnt                  62090 non-null  float64
 4   latitude                      62090 non-null  float64
 5   longitude                     62090 non-null  float64
 6   poolcnt                       62090 non-null  float64
 7   roomcnt                       62090 non-null  float64
 8   threequarterbathnbr           62090 non-null  float64
 9   unitcnt                       62090 non-null  float64
 10  numberofstories               62090 non-null  float64
 11  house_age                     62090 non-null  float64
 12  airconditioningtypeid_1.0     62090 non-null  float64
 13  b

array([-0.04060104,  0.00803361,  0.04714126, ..., -0.02798657,
        0.03477351,  0.00171659])

In [9]:
# automatic parameters tuning
dt=DecisionTreeRegressor()
properties={
            'min_samples_leaf':[x for x in range(2,251,2)],
            'max_leaf_nodes':[x for x in range(2,51,2)],
            'max_features': ["sqrt", "log2"]
            }

tuned_dt=GridSearchCV(dt,properties,scoring="neg_mean_squared_error",cv=5,return_train_score=True,verbose=2,n_jobs=-1)
tuned_dt.fit(engineered_dataset.x_train,engineered_dataset.y_train)

print ("Best Score: {:.3f}".format(tuned_dt.best_score_) )
print ("Best Params: ", tuned_dt.best_params_)

Fitting 5 folds for each of 6250 candidates, totalling 31250 fits
[CV] END max_features=sqrt, max_leaf_nodes=2, min_samples_leaf=2; total time=   0.4s
[CV] END max_features=sqrt, max_leaf_nodes=2, min_samples_leaf=2; total time=   0.3s
[CV] END max_features=sqrt, max_leaf_nodes=2, min_samples_leaf=6; total time=   0.1s
[CV] END max_features=sqrt, max_leaf_nodes=2, min_samples_leaf=6; total time=   0.1s
[CV] END max_features=sqrt, max_leaf_nodes=2, min_samples_leaf=8; total time=   0.1s
[CV] END max_features=sqrt, max_leaf_nodes=2, min_samples_leaf=10; total time=   0.1s
[CV] END max_features=sqrt, max_leaf_nodes=2, min_samples_leaf=10; total time=   0.1s
[CV] END max_features=sqrt, max_leaf_nodes=2, min_samples_leaf=12; total time=   0.2s
[CV] END max_features=sqrt, max_leaf_nodes=2, min_samples_leaf=14; total time=   0.1s
[CV] END max_features=sqrt, max_leaf_nodes=2, min_samples_leaf=16; total time=   0.1s
[CV] END max_features=sqrt, max_leaf_nodes=2, min_samples_leaf=18; total time= 

In [ ]:
# save results

# to load previously saved results


In [10]:
print(tuned_dt.cv_results_)
tuned_dt_results = pd.DataFrame( tuned_dt.cv_results_ )

{'mean_fit_time': array([0.34779062, 0.15273366, 0.08549833, ..., 0.10964994, 0.1493176 ,
       0.10797853]), 'std_fit_time': array([0.04637889, 0.09358142, 0.0268433 , ..., 0.00949364, 0.0187001 ,
       0.01049973]), 'mean_score_time': array([0.01517739, 0.00859494, 0.01307235, ..., 0.00671597, 0.00578842,
       0.00480833]), 'std_score_time': array([0.01156354, 0.00433457, 0.00776415, ..., 0.001035  , 0.00070905,
       0.00082017]), 'param_max_features': masked_array(data=['sqrt', 'sqrt', 'sqrt', ..., 'log2', 'log2', 'log2'],
             mask=[False, False, False, ..., False, False, False],
       fill_value='?',
            dtype=object), 'param_max_leaf_nodes': masked_array(data=[2, 2, 2, ..., 50, 50, 50],
             mask=[False, False, False, ..., False, False, False],
       fill_value='?',
            dtype=object), 'param_min_samples_leaf': masked_array(data=[2, 4, 6, ..., 246, 248, 250],
             mask=[False, False, False, ..., False, False, False],
       fill_valu

In [11]:
tuned_dt_results[["mean_test_score","mean_train_score"]]

,mean_test_score,mean_train_score
0,-0.028929,-0.028918
1,-0.028939,-0.028910
2,-0.028970,-0.028880
3,-0.028926,-0.028888
4,-0.028952,-0.028897
...,...,...
6245,-0.028976,-0.028676
6246,-0.028978,-0.028660
6247,-0.028984,-0.028688
6248,-0.028954,-0.028654


In [12]:
print(tuned_dt.best_estimator_)
print(tuned_dt.best_score_)

DecisionTreeRegressor(max_features='sqrt', max_leaf_nodes=14,
                      min_samples_leaf=236)
-0.02887239542624741


In [13]:
test_acc = mean_squared_error(y_true = engineered_dataset.y_train,
                          y_pred = tuned_dt.predict(engineered_dataset.x_train) )
print ("Train Accuracy: {}".format(test_acc) )

Train Accuracy: 0.028795857775940466


In [9]:
# best parameters from automatic parameters tuning
tree_reg = DecisionTreeRegressor(max_features='sqrt', max_leaf_nodes=14, min_samples_leaf=236)
tree_reg.fit(engineered_dataset.x_train, engineered_dataset.y_train)

DecisionTreeRegressor(max_features='sqrt', max_leaf_nodes=14,
                      min_samples_leaf=236)

In [10]:
dtreg_train_pred = tree_reg.predict(engineered_dataset.x_train)
dtreg_test_pred = tree_reg.predict(engineered_dataset.x_test)

dtreg_train_eval = eva.Evaluation(y_real=engineered_dataset.y_train, y_pred=dtreg_train_pred)
dtreg_test_eval = eva.Evaluation(y_real=engineered_dataset.y_test, y_pred=dtreg_test_pred)

In [11]:
print("Training:")
dtreg_train_eval.print_eval()
print("Testing:")
dtreg_test_eval.print_eval()

Training:
--------------Model Evaluations:--------------
Mean Absolute Error : 0.07040338586908021
Mean Squared Error : 0.02878721326104192
Root Mean Squared Error : 0.16966795001131452

Testing:
--------------Model Evaluations:--------------
Mean Absolute Error : 0.0722301065071152
Mean Squared Error : 0.030227355962183296
Root Mean Squared Error : 0.1738601620906391



# AdaBoost Regression Model

-------------DA FARE BENE-----------
vedere notebook lezione su che parametri ci sono da tunare perche lotto lo fa

In [29]:
# automatic parameters tuning
abreg=AdaBoostRegressor()
properties={
    'base_estimator':[tree_reg],
    'n_estimators':[x for x in range(1,50,2)],
    'learning_rate':[0.001,0.01,0.1,1,10,100]
}

tuned_abreg=GridSearchCV(abreg,properties,scoring="neg_mean_squared_error",cv=5,return_train_score=True,verbose=4,n_jobs=-1)
tuned_abreg.fit(engineered_dataset.x_train,engineered_dataset.y_train)

print ("Best Score: {:.3f}".format(tuned_abreg.best_score_) )
print ("Best Params: ", tuned_abreg.best_params_)

Fitting 5 folds for each of 150 candidates, totalling 750 fits
[CV 2/5] END base_estimator=DecisionTreeRegressor(max_features='sqrt', max_leaf_nodes=14,
                      min_samples_leaf=236), learning_rate=0.001, n_estimators=1;, score=(train=-0.029, test=-0.028) total time=   0.7s
[CV 2/5] END base_estimator=DecisionTreeRegressor(max_features='sqrt', max_leaf_nodes=14,
                      min_samples_leaf=236), learning_rate=0.001, n_estimators=3;, score=(train=-0.029, test=-0.028) total time=   0.5s
[CV 1/5] END base_estimator=DecisionTreeRegressor(max_features='sqrt', max_leaf_nodes=14,
                      min_samples_leaf=236), learning_rate=0.001, n_estimators=5;, score=(train=-0.028, test=-0.030) total time=   1.2s
[CV 5/5] END base_estimator=DecisionTreeRegressor(max_features='sqrt', max_leaf_nodes=14,
                      min_samples_leaf=236), learning_rate=0.001, n_estimators=5;, score=(train=-0.029, test=-0.029) total time=   1.1s
[CV 4/5] END base_estimator=Decis

In [ ]:
# save results

# to load previously saved results


In [30]:
print(tuned_abreg.cv_results_)
tuned_abreg_results = pd.DataFrame( tuned_abreg.cv_results_ )

{'mean_fit_time': array([ 0.53477936,  0.63662601,  1.08942022,  1.37055883,  2.38012795,
        3.4077137 ,  3.8485764 ,  2.24673672,  3.10827703,  7.73846602,
        6.06521497,  4.9561718 ,  4.9906374 ,  3.94031658,  3.95685434,
        4.67131858,  5.60035911, 11.99625311, 10.22415032,  6.11465945,
        5.38704081,  5.84188976,  6.90521426,  6.29700065,  6.30802236,
        0.19894838,  0.45436215,  0.76856771,  1.09962692,  1.61799703,
        2.74138427,  2.06057296,  2.77513485,  3.08100891,  2.28152084,
        2.88263435,  2.78703794,  3.40284185,  3.85348406,  3.77594061,
        5.33809109,  5.50941105,  4.78544812,  4.91492443,  5.91200032,
        6.37249575,  6.440346  ,  5.8511941 ,  5.88512378,  6.33429999,
        0.15083337,  0.36856308,  0.79156351,  1.08672261,  1.20251536,
        1.82346897,  2.01887774,  2.63846245,  3.26309099,  2.65737519,
        2.78539853,  2.87657285,  3.72657199,  5.03095784,  3.98378773,
        5.47576318,  6.61282105,  8.01892357, 

In [31]:
tuned_abreg_results[["param_n_estimators","mean_test_score","mean_train_score"]]

,param_n_estimators,mean_test_score,mean_train_score
0,1,-0.029021,-0.028851
1,3,-0.028877,-0.028788
2,5,-0.028892,-0.028777
3,7,-0.028871,-0.028768
4,9,-0.028873,-0.028770
...,...,...,...
145,41,-3.100276,-3.100371
146,43,-3.105351,-3.105722
147,45,-3.205980,-3.206028
148,47,-3.276212,-3.276377


In [32]:
print(tuned_abreg.best_estimator_)
print(tuned_abreg.best_score_)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_features='sqrt',
                                                       max_leaf_nodes=14,
                                                       min_samples_leaf=236),
                  learning_rate=0.001, n_estimators=13)
-0.02885277883487413


In [33]:
test_acc = mean_squared_error(y_true = engineered_dataset.y_train,
                          y_pred = tuned_abreg.predict(engineered_dataset.x_train) )
print ("Train Accuracy: {}".format(test_acc) )

Train Accuracy: 0.028780691690925734


In [16]:
# best parameters from automatic parameters tuning
adaboost_reg = AdaBoostRegressor(base_estimator=tree_reg,n_estimators=13,learning_rate=0.001)
adaboost_reg.fit(engineered_dataset.x_train, engineered_dataset.y_train)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_features='sqrt',
                                                       max_leaf_nodes=14,
                                                       min_samples_leaf=236),
                  learning_rate=0.001, n_estimators=13)

In [17]:
adaboostreg_train_pred = adaboost_reg.predict(engineered_dataset.x_train)
adaboostreg_test_pred = adaboost_reg.predict(engineered_dataset.x_test)

adaboostreg_train_eval = eva.Evaluation(y_real=engineered_dataset.y_train, y_pred=adaboostreg_train_pred)
adaboostreg_test_eval = eva.Evaluation(y_real=engineered_dataset.y_test, y_pred=adaboostreg_test_pred)

In [18]:
print("Training:")
adaboostreg_train_eval.print_eval()
print("Testing:")
adaboostreg_test_eval.print_eval()

Training:
--------------Model Evaluations:--------------
Mean Absolute Error : 0.0699778024186964
Mean Squared Error : 0.028785121077958022
Root Mean Squared Error : 0.1696617843769127

Testing:
--------------Model Evaluations:--------------
Mean Absolute Error : 0.07168050169535041
Mean Squared Error : 0.03014750072160507
Root Mean Squared Error : 0.17363035656706194



# Random Forest Regression Model

-------SCRIVERE QUALCOSA----------------

fare GridSearchCV per il tuning di parametri cosi ho un motivo per cui sto usando certi parametri

uso parametri trovati per decision tree

In [ ]:
RF_HYPER_PARAMS = { # riempire con risultati del gridsearchsv
    "n_estimators": [150, 200, 250],
    "min_samples_leaf": [1, 50, 100, 200],
    "max_leaf_nodes": [2, 5, 10],
    "max_features": ["sqrt", "log2"]
} saltato

In [53]:
rf_reg = RandomForestRegressor()

tuned_model_rf = GridSearchCV(rf_reg, RF_HYPER_PARAMS, cv=5, verbose=2,n_jobs=-1, scoring="neg_mean_squared_error")
tuned_model_rf.fit(engineered_dataset.x_train.values, engineered_dataset.y_train)

print("Tuning results:")
print(f"Best params: {tuned_model_rf.best_params_}")

print("Fitting with best params and full training set...")
tuned_rf = RandomForestRegressor(n_jobs=-1, **tuned_model_rf.best_params_)
tuned_rf.fit(engineered_dataset.x_train, engineered_dataset.y_train)

# evaluate

RandomForestRegressor(max_depth=6, n_estimators=50)

In [ ]:
# automatic parameters tuning
rfreg=RandomForestRegressor()
properties={
    "n_estimators": [x for x in range(50,251,50)],
    "min_samples_leaf": [x for x in range(180,251,7)],
    "max_leaf_nodes": [x for x in range(14,71,14)],
    "max_features": ["sqrt"]
}

tuned_rfreg=GridSearchCV(rfreg,properties,scoring="neg_mean_squared_error",cv=5,return_train_score=True,verbose=4,n_jobs=-1)
tuned_rfreg.fit(engineered_dataset.x_train,engineered_dataset.y_train)

print ("Best Score: {:.3f}".format(tuned_rfreg.best_score_) )
print ("Best Params: ", tuned_rfreg.best_params_)

Fitting 5 folds for each of 275 candidates, totalling 1375 fits
[CV 1/5] END max_features=sqrt, max_leaf_nodes=14, min_samples_leaf=180, n_estimators=50;, score=(train=-0.028, test=-0.030) total time=   9.5s
[CV 1/5] END max_features=sqrt, max_leaf_nodes=14, min_samples_leaf=180, n_estimators=100;, score=(train=-0.028, test=-0.030) total time=  17.9s
[CV 5/5] END max_features=sqrt, max_leaf_nodes=14, min_samples_leaf=180, n_estimators=100;, score=(train=-0.029, test=-0.029) total time=  19.3s
[CV 4/5] END max_features=sqrt, max_leaf_nodes=14, min_samples_leaf=180, n_estimators=150;, score=(train=-0.029, test=-0.030) total time=  22.5s
[CV 3/5] END max_features=sqrt, max_leaf_nodes=14, min_samples_leaf=180, n_estimators=200;, score=(train=-0.029, test=-0.027) total time=  27.3s
[CV 2/5] END max_features=sqrt, max_leaf_nodes=14, min_samples_leaf=180, n_estimators=250;, score=(train=-0.029, test=-0.027) total time=  31.3s
[CV 1/5] END max_features=sqrt, max_leaf_nodes=14, min_samples_leaf

In [ ]:
# save results

# to load previously saved results

type(tuned_rfreg) # cosi so come salvarlo, poi lo faccio anche per lin reg, dt e adaboost

In [ ]:
print(tuned_rfreg.cv_results_)
tuned_rfreg_results = pd.DataFrame( tuned_rfreg.cv_results_ )

In [ ]:
tuned_rfreg_results[["param_n_estimators","mean_test_score","mean_train_score"]]

In [ ]:
print(tuned_rfreg.best_estimator_)
print(tuned_rfreg.best_score_)

In [ ]:
test_acc = mean_squared_error(y_true = engineered_dataset.y_train,
                          y_pred = tuned_rfreg.predict(engineered_dataset.x_train) )
print ("Train Accuracy: {}".format(test_acc) )

In [ ]:
# best parameters from automatic parameters tuning
forest_reg = RandomForestRegressor(n_estimators=2,min_samples_leaf=2,max_leaf_nodes=2,max_features="sqrt")
forest_reg.fit(engineered_dataset.x_train, engineered_dataset.y_train)

In [ ]:
rfreg_train_pred = forest_reg.predict(engineered_dataset.x_train)
rfreg_test_pred = forest_reg.predict(engineered_dataset.x_test)

rfreg_train_eval = eva.Evaluation(y_real=engineered_dataset.y_train, y_pred=rfreg_train_pred)
rfreg_test_eval = eva.Evaluation(y_real=engineered_dataset.y_test, y_pred=rfreg_test_pred)

In [ ]:
print("Training:")
rfreg_train_eval.print_eval()
print("Testing:")
rfreg_test_eval.print_eval()

# Performance Visualization

In [ ]:
che ci faccio qua?

In [30]:
def model_selection():
    """
    Funzione che seleziona il modello migliore tra quelli analizzati.

    Returns:
        model_min (Object): modello migliore.
    """
    models=[best_AdaBoostRegressor,best_DecisionTreeRegressor,best_LinearRegressor,best_randomForestRegressor]

    min_mse=None
    model_min=None

    for i,m in enumerate(models):

        m.fit(train_X,train_Y["logerror"])
        y_pred=m.predict(test_X)
        mse=mean_squared_error(test_Y["logerror"],y_pred)

        if i==0:
            min_mse=mse
            model_min=m

        print(str(i)+") ","model:",m,"\n    mse:",mse,'\n')

        if min_mse > mse:
            min_mse=mse
            model_min=m

    return model_min

In [31]:
best=model_selection()
print("modello migliore: ",best)

0)  model: AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_features=0.95,
                                                       max_leaf_nodes=340),
                  learning_rate=0.0075, n_estimators=25) 
    mse: 0.02689255679969093 

1)  model: DecisionTreeRegressor(max_features=0.95, max_leaf_nodes=340) 
    mse: 0.026993166052583947 

2)  model: LinearRegression(normalize=True, positive=True) 
    mse: 0.02794294912909073 

3)  model: RandomForestRegressor(n_estimators=50) 
    mse: 0.014583626349850104 

modello migliore:  RandomForestRegressor(n_estimators=50)


pier:

In [ ]:
def get_performance_df(results: dict[str, list[ev.RegressorEvaluation]]):
    perf_records = []
    for model_name, evaluations in results.items():
        for evl in evaluations:
            record = {
                "model": model_name,
                "dataset id": evl.regressor_id,
                "MAE": evl.mae,
                "MSE": evl.mse,
                "R2": evl.r2
            }
            perf_records.append(record)

    return pd.DataFrame.from_records(data=perf_records).sort_values(by="dataset id")

def performance_plot(performance_df: pd.DataFrame):
    plot = sns.lineplot(data=performance_df, x="dataset id", y="MSE", hue="model",
                        style="model", palette="pastel", markers=True)
    plot.tick_params(axis="x", rotation=90)

    return plot

In [ ]:
def plot_features_vs_predictions(evaluation: ev.RegressorEvaluation):
    dataset_name = evaluation.regressor_id
    dataset = engineered_datasets[dataset_name]
    test_data = dataset.x_test

    fig, axs = vis.bivariate_feature_plot(data=test_data, mode="scatter",
                                          y_var=("Model predictions", pd.Series(evaluation.y_pred)),
                                          subplot_size=(5, 4),
                                          width=3, title_size=50,
                                          title=f"[{dataset_name}] Features vs Predictions",
                                          scatter_kwargs={
                                              "alpha": 0.8
                                          })

    return fig, axs


def plot_features_vs_residuals(evaluation: ev.RegressorEvaluation):
    dataset_name = evaluation.regressor_id
    dataset = engineered_datasets[dataset_name]
    test_data = dataset.x_test

    residuals = evaluation.y_true - evaluation.y_pred
    fig, axs = vis.bivariate_feature_plot(data=test_data, mode="scatter",
                                          y_var=("Model residuals", pd.Series(residuals)),
                                          subplot_size=(5, 4),
                                          width=3, title_size=50,
                                          title=f"[{dataset_name}] Features vs Residuals",
                                          scatter_kwargs={
                                              "alpha": 0.8
                                          })

    return fig, axs


def get_extreme_predictions(data: pd.DataFrame, evaluation: ev.RegressorEvaluation, percentile: float):
    most_wrong = ev.get_highest_error_instances(data=data, percentile=percentile,
                                                pred=evaluation.y_pred, true_pred=evaluation.y_true,
                                                error_type="squared")
    most_correct = ev.get_lowest_error_instances(data=data, percentile=percentile,
                                                 pred=evaluation.y_pred, true_pred=evaluation.y_true,
                                                 error_type="squared")

    return most_wrong, most_correct

In [ ]:
color_red = "#bf1515"
color_green = "#32a852"

def plot_extreme_instances_on_distribution(evaluation: ev.RegressorEvaluation):
    dataset_name = evaluation.regressor_id
    dataset = engineered_datasets[dataset_name]
    test_data = dataset.x_test.copy()

    fig, axs = vis.feature_distributions_plot(data=test_data, numerical_mode="violin",
                                              subplot_size=(5, 4),
                                              width=5, title_size=40,
                                              title=f"[{dataset_name}] Most Wrong/Correct on Distributions")

    most_wrong, most_correct = get_extreme_predictions(data=test_data,
                                                       evaluation=evaluation,
                                                       percentile=99.5)
    for ax in axs.flatten():
        feature_name = ax.get_xlabel()
        if feature_name != "":
            x_worst = most_wrong[feature_name].values
            x_best = most_correct[feature_name].values

            # Points are drawn at mid height + an offset so they don't overlap
            y_min, y_max = ax.get_ylim()
            height = (abs(y_max) - abs(y_min))
            half_height = height / 2

            # Heights of worst and best on two different levels
            y_worst_height = half_height + 1
            y_best_height = half_height + 2

            # Also add gaussian noise to mitigate overlapping with the violin/box plot
            y_worst = [y_worst_height + np.random.normal(0, 0.05) for _ in range(len(x_worst))]
            y_best = [y_best_height + np.random.normal(0, 0.05) for _ in range(len(x_best))]

            # Plot the most wrong/correct values over the distribution plots
            # and assign them size in proportion to their wrongness/correctness
            worst_size = 50 * ((np.argsort(most_wrong["errors"].values) + 1) / len(x_worst))
            best_size = 50 * ((np.argsort(-most_correct["errors"].values) + 1) / len(x_best))

            ax.scatter(x=x_worst, y=y_worst, s=worst_size, c=color_red)
            ax.scatter(x=x_best, y=y_best, s=best_size, c=color_green)

    return fig, axs

In [ ]:
def plot_extreme_instances_on_feature_vs_target(evaluation: ev.RegressorEvaluation):
    dataset_name = evaluation.regressor_id
    dataset = engineered_datasets[dataset_name]
    test_data = dataset.x_test

    fig, axs = vis.bivariate_feature_plot(data=test_data, mode="scatter",
                                          y_var=("True logerror", pd.Series(dataset.y_test)),
                                          subplot_size=(5, 4),
                                          width=3, title_size=40,
                                          title=f"[{dataset_name}] Most Wrong/Correct on Features vs Target",
                                          scatter_kwargs={
                                              "alpha": 0.65  # so that extreme instances are highlighted
                                          })

    most_wrong, most_correct = get_extreme_predictions(data=test_data,
                                                       evaluation=evaluation,
                                                       percentile=99.5)
    for ax in axs.flatten():
        feature_name = ax.get_xlabel()
        if feature_name != "":
            x_worst = most_wrong[feature_name].values
            x_best = most_correct[feature_name].values

            y_worst = most_wrong["true predictions"].values
            y_best = most_correct["true predictions"].values

            # Plot the most wrong/correct values over the distribution plots
            # and assign them size in proportion to their wrongness/correctness
            worst_size = 80 * ((np.argsort(most_wrong["errors"].values) + 1) / len(x_worst))
            best_size = 80 * ((np.argsort(-most_correct["errors"].values) + 1) / len(x_best))

            ax.scatter(x=x_worst, y=y_worst, s=worst_size, c=color_red)
            ax.scatter(x=x_best, y=y_best, s=best_size, c=color_green)

    return fig, axs

In [ ]:
# all results
training_evals = {
    "[Training] Linear Regression": linreg_train_eval,
    "[Training] Decision Tree Regression": dtreg_train_eval,
    "[Training] AdaBoost Regression": adaboostreg_train_eval,
    "[Training] Random Forest Regression": rfreg_train_eval
}

testing_evals = {
    "[Testing] Linear Regression": linreg_test_eval,
    "[Testing] Decision Tree Regression": dtreg_test_eval,
    "[Testing] AdaBoost Regression": adaboostreg_test_eval,
    "[Testing] Random Forest Regression": rfreg_test_eval
}

## Training Performance

In [ ]:
train_performance_df = get_performance_df(training_results)
train_performance_df

In [ ]:
performance_plot(train_performance_df)

## Testing Performance

In [ ]:
test_performance_df = get_performance_df(testing_results)
test_performance_df

In [ ]:
performance_plot(test_performance_df)

## Training vs Testing

In [ ]:
train_test_perf_df = pd.concat([train_performance_df, test_performance_df]).reset_index(drop=True)
train_test_perf_df = train_test_perf_df.sort_values(by="dataset id")
train_test_perf_df

In [ ]:
performance_plot(train_test_perf_df)

## Predictions and Residuals

In this section, for each combination of model and testing set, 2 types of plots will be shown:
- Features vs model predictions;
- Features vs model residuals.

Here the goal is to understand how the models make their predictions and get a general idea of where and how wrong they are.

In [ ]:
la parte di lotto sta piu sotto invece che in questo punto come pier

### Linear Regression

In [ ]:
for result in lin_reg_test_results:
    plot_features_vs_predictions(evaluation=result)
    plot_features_vs_residuals(evaluation=result)

### Decision Forest Regression

### AdaBoost Regression

### Random Forest Regression

In [ ]:
for result in rf_reg_test_results:
    plot_features_vs_predictions(evaluation=result)
    plot_features_vs_residuals(evaluation=result)

### A Look at the Expected Value and Variance of True log-errors

The expected value of both training and testing sets' targets is shown to provide a better context around the previously plotted predictions. We can see that the expected value is around 0.017, which is very close to what the models are predicting: in other words, the models seem to be predicting values around the average of the true log-errors with relatively little variance.

A look at the variance of true log-errors also gives an idea as to why the testing sets perform much better than the training ones: since all the models seem to predict the average log-error, or very close to it, for each instance, error is expected to be directly proportional with the variance of the target of each set. The variance of the testing set is, in fact, lower than the training set one, most likely because its size is smaller.

In [ ]:
# Since all y_train and y_test are equal, the dataset from which they are extracted
# does not matter
any_dataset = "lvl1-leave-one-out"
y_train = engineered_datasets[any_dataset].y_train
y_test = engineered_datasets[any_dataset].y_test

pd.DataFrame(data={
    "Set": ["Training", "Testing"],
    "Expected value": [y_train.mean(), y_test.mean()],
    "Variance": [y_train.var(), y_test.var()]
})

## Best and Worst Instances

In this section, for each combination of model and testing set, 2 types of plots will be shown:
- Distribution of extreme instances (in terms of predictions) vs actual feature distribution;
- Dataset features and extreme instances vs true logerror.

Both plots' goal is to help me understand if there is some peculiarity in the distribution and predictions of the most wrongly/correctly predicted instances.

In [ ]:
lotto ha un analisi delle istanze come pier

### Linear Regression

In [ ]:
for result in lin_reg_test_results:
    plot_extreme_instances_on_distribution(evaluation=result)
    plot_extreme_instances_on_feature_vs_target(evaluation=result)

### Decision Forest Regression

### AdaBoost Regression

### Random Forest Regression

In [ ]:
for result in rf_reg_test_results:
    plot_extreme_instances_on_distribution(evaluation=result)
    plot_extreme_instances_on_feature_vs_target(evaluation=result)

----------------------- DA NOTEBOOK CRASTO -----------------------

## Model Evaluation

### Baseline Metrics

It is important to set a baseline for the model's performance to compare different algorithms. For regression problems, the baseline metrics are calculated by replacing $y'$ with $\bar{y}$. Using this, the different baseline regression metrics are:

- **MSE Baseline**: Variance of the target variable (Mean Squared Error)
- **RMSE Baseline**: Standard Deviation of the target variable (Root Mean Squared Error)
- **MAE Baseline**: Average Abolsute Deviation of the target variable (Mean Absolute Error)
- **R2 Baseline**: 0

For this regression problem, we will use the models' **Mean Absolute Error** and **RMSE (Root Mean Squared Error)** to compare the different algorithms which have **baseline values of 0.533 and 0.0837** respectively.

We will also observe the RMSE as another evaluation metric which punishes more for outliers than MAE.

In [ ]:
# Baseline for RMSE
print(f"MAE Baseline: {engineered_dataset.y_train.mad()}")
print(f"RMSE Baseline: {engineered_dataset.y_train.std()}")

### MAE Evaluation

To evaluate and short list the most promising models, we will use the models' **MAE** in two different ways:

1) **MAE on Validation Set**: Calculates the MAE on the validation set which is quicker to calculate than evaluation using Cross-Validation. However, it is possible the MAE obtained is skewed depending on the instances sampled in the validation set.

2) A great alternative is to use **K-Fold Cross-Validation** where the training set is randomly split into `n` subsets (for example 10 subsets) called *folds*. It trains and evaluates the model 10 times, picking a different fold for evaluation every time and training on the other 9 folds. Result is an array containing the 10 evaluation scores. Takes longer to evaluate but provides a more accurate measure of the model's performance.

questo sotto e da provare

In [24]:
def display_scores(model, scores):
    print("-"*50)
    print("Model:", model)
    print("\nScores:", scores)
    print("\nMean:", scores.mean())
    print("\nStandard deviation:", scores.std())

def get_cross_val_scores(models, X, y, cv=10, fit_params=None):
    """
    Performs k-fold cross validation and calculates MAE for each fold for all input models.
    `models`: list of fit models
    """
    for model in models:
        mae = -cross_val_score(model, X, y, scoring="neg_mean_absolute_error", cv=cv, fit_params=fit_params)
        display_scores(model, mae)

    # Test usage of cross val function
# get_cross_val_scores([lin_reg, ridge_reg], X_prepared, y_train, cv=5)

# Conclusion

1. I have performed all the feature engineering steps necessary to ensure the dataset is ready to be fed into Machine Learning algorithms.

2. After Pre-processing and Feature Engineering the raw dataset we splitted the dataset into train and test sets.

3. Performed Feature scaling on data for better performance.

4. Trained multiple models using different ML regression algorithms on dataset.

5. Appleied Performance metrics such as MAE, MSE, RMSE to find out best prediction model.

6. With the help of GridSearch CV we found out best estimator with least Root mean squred error.

7. Saved best predictor in .pickle format for future predictions.

8. Done prediction on test data and saved predictions into .csv file.

### Learning Curves  (solo per linear regression)

To detect where the model is under or overfitting, we can look at the *learning curves*. **Learning Curves** are plots of the model’s performance on the training set and the validation set as a function of the training set size (or the training iteration). To generate the plots, train the model several times on different sized subsets of the training set.


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

def plot_learning_curves(model, X_train, y_train, X_val, y_val):
    """
    Train the input model on different sized subsets and test on validation set.
    Output a plot of training and validation error for the different sized subsets.
    """
    train_errors, val_errors = [], []
    num_instances = np.linspace(1, len(X_train), num=15).astype(int)

    for m in num_instances:
        model.fit(X_train[:m], y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_val_predict = model.predict(X_val)
        train_errors.append(mean_absolute_error(y_train[:m], y_train_predict))
        val_errors.append(mean_absolute_error(y_val, y_val_predict))
    plt.plot(num_instances, train_errors, "r-+", linewidth=2, label="train")
    plt.plot(num_instances, val_errors, "b-", linewidth=3, label="val")
    plt.legend(loc='best')
    plt.title(model)
    return plt

In [ ]:
lin_reg = LinearRegression()
plt = plot_learning_curves(lin_reg, X_prepared, y_train, X_prepared_val, y_val)
plt.ylim(0, 0.25)
plt.show()

### Learning Curve Conclusion for Linear Regression

- `train` curve starts with really low RMSE due to the model overfitting on the small number of instances. Conversely, `val` curve (highest value cut off) starts with an extremely high RMSE which is expected since the model is overfit on the small number of training instances.
- The Linear Regression model is underfitting as increasing the number of instances does not improve the model's performance much on either dataset.
- Furthermore, the model is not overfitting as the MAE error observed for both the validation and training sets is extremely close (errors would be different between `train` and `val` datasets if the model were overfitting with the `train` dataset, thus showing lower MAE values.

**As expected, Simple Linear Regression is unable to handle the intricacies of the training data and thus is a high bias model which is underfitting.**